#### List of IoT Sensors:

- GPS Sensor: To track the real-time location of the vehicle.
- Passenger Counting Sensor: To count the number of passengers entering and exiting the vehicle.
- Internet Connectivity Module: Cellular or Wi-Fi module to enable communication with the transit information platform.

#### Python Script:

- The python script is used to simulate the IoT sensors and generate the data.

##### To connect to the MQTT Broker:
- Start the MQTT Broker on the local machine.
- Run the python script to connect to the MQTT Broker.
- The python script will publish the data to the MQTT Broker.
- Subscribe to the MQTT Broker to receive the data.
- Command : `mosquitto_sub -t pto_data -d -h localhost`

##### Fetch Data from Arduino using MQTT Broker and run Server with Flask:

In [5]:
import json
import paho.mqtt.client as mqtt
from flask import Flask, render_template, jsonify
from flask_cors import CORS

app = Flask("PTO_Backend")
CORS(app)

data = {}  # Initialize data variable

def on_message(client, userdata, message):
    global data
    data = json.loads(message.payload.decode("utf-8"))
    print("Received message:", data)

mqtt_client = mqtt.Client(protocol=mqtt.MQTTv5)
mqtt_client.on_message = on_message

try:
    mqtt_client.connect("192.168.29.170", 1883)
    topic = "pto_data"
    mqtt_client.subscribe(topic)
    print("Subscribed to topic:", topic)
    mqtt_client.loop_start()  # Start the MQTT client loop

except Exception as e:
    print(f"Error: {str(e)}")
    mqtt_client.disconnect()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/data', methods=['GET'])
def get_data():
    return jsonify(data)

if __name__ == "__main__":
    app.run(port=5500, threaded=True)

Subscribed to topic: pto_data
 * Serving Flask app 'PTO_Backend'
 * Debug mode: off


 * Running on http://127.0.0.1:5500
Press CTRL+C to quit
